# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [ ]:
CREATE VIEW Metrics AS
SELECT I.fcid_code code, COUNT(DISTINCT seqn) popularidade, SUM(I.intake) intake_sum,
AVG(I.intake) avg_intake, AVG(I.intake_bw) avg_bw, COUNT(DISTINCT R.food_code) receitas
FROM INTAKE I, RECIPES R, FCID_DESCRIPTION F
WHERE R.fcid_code = I.fcid_code
GROUP BY I.fcid_code
ORDER BY popularidade DESC;

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [ ]:
/*Alimentos que as pessoas mais escolhem:*/
SELECT M.code
FROM Metrics M
ORDER BY M.popularidade DESC 

/*Alimentos  mais consumidos em quantidade:*/
SELECT M.code
FROM Metrics M
ORDER BY M.intake_sum DESC

/*É interessante notar que os produtos mais escolhidos por pessoas 
não necessariamente serão aqueles consumidos em maior quantidade. 
Logo, é importante ponderar ambos os fatores na análise. 
Isto é, o produto mais popular é aquele que, por exemplo, 
teria a maior relação quantidade/aparição. 
Isso poderia ser feito da seguinte forma:*/

SELECT M.code
FROM Metrics M
ORDER BY M.intake_sum/M.popularidade DESC;


## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

In [ ]:
Podemos analisar os consumidores conforme diferentes perfis. 
Por exemplo, podemos agrupar os consumidores pelos alimentos que mais consomem; 
agrupar por aqueles que são veganos; agrupar por aqueles que não comem salada; 
agrupar por aqueles que consomem mais alimentos variados, dentre outros.
Com esses grupos em mãos, poderíamos ter diferentes classes. 
Se dividíssemos os consumidores em grupos dos alimentos que mais consomem, 
teríamos classes do tipo:
Classe dos que consomem carne, Classe dos que consomem arroz. 
Dentro da classe dos que consomem carne, teríamos os grupos dos que comem mais carne bovina,
dos que consomem mais frango e dos que consomem mais carne suína, etc. 
E dentro da Classe dos que consomem arroz teríamos o grupo dos que comem mais arroz integral,
grupo dos que consomem mais arroz biro-biro, etc. 
Assim estabelecemos relações entre os grupos e classes.

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [ ]:
/*Vamos criar uma query que relaciona o nome da comida com o peso médio das pessoas que o consomem
e com a popularidade da comida. Assim, determinamos quais alimentos tendem a causar 
maior aumento de peso e também o peso atrelado aos alimentos mais populares, ou seja, 
avaliamos se a popularidade do alimento implica em ele contribuir ou não 
para um ganho de peso.*/


SELECT M.code, (M.avg_bw/M.avg_intake) mass , (M.intake_sum/M.popularidade) abs_popularity
FROM METRICS M
ORDER BY mass DESC;